In [ ]:
# Listers les PDFs raw_pdf/
# Convertir les PDF en texte raw_text/
# Retirer la conclusions dans processed/
# Faire de CSV contenant toute les info dans processed/
# Appli pigeon pour annotation à partir de chaque raw_text qui ajoute le label